Our goal is to see if using candle sticks information and information derived from data could be used to predict wheather the price will go up or down so we could execute put and call options.our target is to make 51% correct prediction(offcourse in reality we will need more than 51% predictions to make our selves profitable )
For this model we will try to test an investment of which the price does not depent on the season,press release or a single tweet  but only bullish and bearish activies in market
we will first test with different different expiry dates of options from 1min  to 1 week.we will adjust the granularity of the data.
We will start with basics modles than will build on that.we will add advanced calculations and then well will try to integrate out results with knowledge base from various books and experts.



In [83]:
!apt-get update -y
!apt-get install openjdk-11-jdk -y


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,400 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,818 kB]
Fetched 12.2 MB in 2s (6,138 kB/s)
Reading package lists... Don

In [84]:
import yfinance as yf

# Example: get 1-minute data for AAPL for the last 5 days
data = yf.download("GLD", interval="1m", period="6d")

print(data.head())


/tmp/ipython-input-115912881.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("GLD", interval="1m", period="6d")
[*********************100%***********************]  1 of 1 completed


Price                           Close        High         Low        Open  \
Ticker                            GLD         GLD         GLD         GLD   
Datetime                                                                    
2025-10-23 13:30:00+00:00  381.230011  381.529999  381.029999  381.459991   
2025-10-23 13:31:00+00:00  380.910004  381.299988  380.799988  381.174500   
2025-10-23 13:32:00+00:00  380.311493  381.049988  380.309998  380.885590   
2025-10-23 13:33:00+00:00  380.834991  380.869995  380.070007  380.359985   
2025-10-23 13:34:00+00:00  380.779999  381.000000  380.549988  380.779999   

Price                      Volume  
Ticker                        GLD  
Datetime                           
2025-10-23 13:30:00+00:00  979979  
2025-10-23 13:31:00+00:00  194381  
2025-10-23 13:32:00+00:00   82744  
2025-10-23 13:33:00+00:00  135811  
2025-10-23 13:34:00+00:00   83654  


In [85]:
pip install -U polygon-api-client

In [86]:
#from polygon import RESTClient

In [87]:
#client = RESTClient(api_key="enter api key")

In [88]:
'''
ticker = "GLD"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2024-01-01", to="2025-01-01", limit=50000):
    aggs.append(a)

print(aggs)
'''
#this data is stored in the txt.We will use the txt to get the data

'\nticker = "GLD"\n\n# List Aggregates (Bars)\naggs = []\nfor a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2024-01-01", to="2025-01-01", limit=50000):\n    aggs.append(a)\n\nprint(aggs)\n'

In [89]:
'''
with open("gold_data for 2024-2025.txt", "w") as file:
    for item in aggs:
        file.write(f"{item}\n")
'''

'\nwith open("gold_data for 2024-2025.txt", "w") as file:\n    for item in aggs:\n        file.write(f"{item}\n")\n'

In [90]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, LongType, IntegerType, StringType


In [91]:
# Read the file line by line
with open("gold_data_for_2024_2025.txt", "r") as file:
    raw_data = file.read().splitlines()  # or use: list(file)

data = []
for s in raw_data:
    # Skip empty lines
    if not s.strip():
        continue

    # Extract key=value pairs
    items = re.findall(r'(\w+)=([-\w.]+|None)', s)

    # Convert to dict
    d = {}
    for k, v in items:
        if v == "None":
            d[k] = None
        else:
            try:
                d[k] = float(v)
            except ValueError:
                d[k] = v
    data.append(d)

# Create PySpark DataFrame
spark = SparkSession.builder.appName("GoldData").getOrCreate()
schema = StructType([
    StructField("open", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("close", DoubleType(), True),
    StructField("volume", DoubleType(), True),        # was LongType()
    StructField("vwap", DoubleType(), True),
    StructField("timestamp", DoubleType(), True),     # was LongType()
    StructField("transactions", DoubleType(), True), # was IntegerType()
    StructField("otc", StringType(), True)
])

# Then create the DataFrame with the schema
df = spark.createDataFrame(data, schema=schema)

df.show()

+------+------+------+------+------+--------+-------------+------------+----+
|  open|  high|   low| close|volume|    vwap|    timestamp|transactions| otc|
+------+------+------+------+------+--------+-------------+------------+----+
|191.61| 192.0|191.61| 192.0| 616.0|191.6739|  1.704186E12|         4.0|NULL|
| 192.2| 192.2| 192.2| 192.2| 515.0|   192.2|1.70418624E12|         6.0|NULL|
|192.25|192.25|192.25|192.25| 500.0|  192.25| 1.7041866E12|         1.0|NULL|
| 192.3| 192.3| 192.3| 192.3| 100.0|   192.3|1.70418666E12|         1.0|NULL|
| 192.3| 192.3| 192.3| 192.3| 250.0|   192.3|1.70418702E12|         3.0|NULL|
|192.32|192.32|192.26|192.26|3010.0|192.2997|1.70418768E12|        12.0|NULL|
| 192.6| 192.6| 192.6| 192.6| 300.0|   192.6|1.70418882E12|         1.0|NULL|
|192.26|192.26|192.26|192.26| 201.0|192.2702|1.70419038E12|         4.0|NULL|
|192.23|192.23|192.23|192.23| 598.0|  192.23|1.70419068E12|         1.0|NULL|
| 192.3| 192.3| 192.3| 192.3| 101.0|192.3002|1.70419146E12|     

In [92]:
null_count = df.filter(df["otc"].isNull()).count()
print(f"Number of null values in 'otc': {null_count}")


Number of null values in 'otc': 130447


In [93]:
num_rows = df.count()
print(f"Number of rows: {num_rows}")


Number of rows: 130447


In [94]:
df = df.drop("otc")

In [95]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col,when,lead
from pyspark.sql.functions import row_number, desc

In [96]:
windowSpec = Window.orderBy("timestamp")
df = df.withColumn(
    "BUY/SELL",
    when(lag("close", 1).over(windowSpec) > col("open"), 1).otherwise(0)
)

df.show()


+------+------+------+------+------+--------+-------------+------------+--------+
|  open|  high|   low| close|volume|    vwap|    timestamp|transactions|BUY/SELL|
+------+------+------+------+------+--------+-------------+------------+--------+
|191.61| 192.0|191.61| 192.0| 616.0|191.6739|  1.704186E12|         4.0|       0|
| 192.2| 192.2| 192.2| 192.2| 515.0|   192.2|1.70418624E12|         6.0|       0|
|192.25|192.25|192.25|192.25| 500.0|  192.25| 1.7041866E12|         1.0|       0|
| 192.3| 192.3| 192.3| 192.3| 100.0|   192.3|1.70418666E12|         1.0|       0|
| 192.3| 192.3| 192.3| 192.3| 250.0|   192.3|1.70418702E12|         3.0|       0|
|192.32|192.32|192.26|192.26|3010.0|192.2997|1.70418768E12|        12.0|       0|
| 192.6| 192.6| 192.6| 192.6| 300.0|   192.6|1.70418882E12|         1.0|       0|
|192.26|192.26|192.26|192.26| 201.0|192.2702|1.70419038E12|         4.0|       1|
|192.23|192.23|192.23|192.23| 598.0|  192.23|1.70419068E12|         1.0|       1|
| 192.3| 192.3| 

In [97]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [98]:
def evalutations(predictions):
    multi_evaluator = MulticlassClassificationEvaluator(labelCol="BUY/SELL", predictionCol="prediction")
    binary_evaluator = BinaryClassificationEvaluator(labelCol="BUY/SELL", rawPredictionCol="rawPrediction")

    accuracy = multi_evaluator.setMetricName("accuracy").evaluate(predictions)
    f1 = multi_evaluator.setMetricName("f1").evaluate(predictions)
    precision = multi_evaluator.setMetricName("weightedPrecision").evaluate(predictions)
    recall = multi_evaluator.setMetricName("weightedRecall").evaluate(predictions)
    auc = binary_evaluator.setMetricName("areaUnderROC").evaluate(predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"AUC: {auc:.4f}")
    return predictions.groupBy("BUY/SELL", "prediction").count()

In [99]:
# Example columns
feature_cols = ["open", "high", "low", "close", "volume", "vwap"]

# Convert the target label to numeric
indexer = StringIndexer(inputCol="BUY/SELL", outputCol="label")
df = indexer.fit(df).transform(df)


In [100]:
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)
df = assembler.transform(df)


In [126]:
train_df, test_df = df_5m.randomSplit([0.7, 0.3], seed=42)


In [127]:
lr = LogisticRegression(featuresCol="features", labelCol="BUY/SELL")
model = lr.fit(train_df)
predictions = model.transform(test_df)

In [128]:
evalutations(predictions).show()

Accuracy: 0.7297
F1 Score: 0.7296
Precision: 0.7298
Recall: 0.7297
AUC: 0.7954
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       1|       1.0|14804|
|       1|       0.0| 5030|
|       0|       0.0|13827|
|       0|       1.0| 5576|
+--------+----------+-----+



In [125]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier

# Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="BUY/SELL", numTrees=100)
model = rf.fit(train_df)
predictions = model.transform(test_df)
evalutations(predictions).show()


Accuracy: 0.5323
F1 Score: 0.4901
Precision: 0.5433
Recall: 0.5323
AUC: 0.5561
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       1|       1.0|16184|
|       0|       1.0|14701|
|       1|       0.0| 3650|
|       0|       0.0| 4702|
+--------+----------+-----+



In [105]:
# OR Gradient-Boosted Trees
gbt = GBTClassifier(featuresCol="features", labelCol="BUY/SELL", maxIter=50)
model = gbt.fit(train_df)
predictions = model.transform(test_df)
evalutations(predictions).show()

Accuracy: 0.5864
F1 Score: 0.4398
Precision: 0.5163
Recall: 0.5864
AUC: 0.5166
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       0|       0.0|22892|
|       1|       0.0|16063|
|       0|       1.0|  168|
|       1|       1.0|  119|
+--------+----------+-----+



In [106]:

predictions.select("BUY/SELL", "label", "prediction", "probability").show(10, truncate=False)


+--------+-----+----------+----------------------------------------+
|BUY/SELL|label|prediction|probability                             |
+--------+-----+----------+----------------------------------------+
|0       |0.0  |0.0       |[0.6211788530509051,0.37882114694909486]|
|1       |1.0  |0.0       |[0.5803777731964707,0.41962222680352934]|
|0       |0.0  |0.0       |[0.592980629585629,0.407019370414371]   |
|1       |1.0  |0.0       |[0.6211788530509051,0.37882114694909486]|
|0       |0.0  |0.0       |[0.6107951583281798,0.38920484167182023]|
|0       |0.0  |0.0       |[0.5803777731964707,0.41962222680352934]|
|0       |0.0  |0.0       |[0.6107951583281798,0.38920484167182023]|
|0       |0.0  |0.0       |[0.6099205532282286,0.3900794467717714] |
|1       |1.0  |0.0       |[0.6211788530509051,0.37882114694909486]|
|0       |0.0  |0.0       |[0.6170940069336699,0.3829059930663301] |
+--------+-----+----------+----------------------------------------+
only showing top 10 rows



part 2

In [107]:
df_5m= df.drop('BUY/SELL','PUT/CALL','label','features','transactions')
df_5m.show()

+------+------+------+------+------+--------+-------------+
|  open|  high|   low| close|volume|    vwap|    timestamp|
+------+------+------+------+------+--------+-------------+
|191.61| 192.0|191.61| 192.0| 616.0|191.6739|  1.704186E12|
| 192.2| 192.2| 192.2| 192.2| 515.0|   192.2|1.70418624E12|
|192.25|192.25|192.25|192.25| 500.0|  192.25| 1.7041866E12|
| 192.3| 192.3| 192.3| 192.3| 100.0|   192.3|1.70418666E12|
| 192.3| 192.3| 192.3| 192.3| 250.0|   192.3|1.70418702E12|
|192.32|192.32|192.26|192.26|3010.0|192.2997|1.70418768E12|
| 192.6| 192.6| 192.6| 192.6| 300.0|   192.6|1.70418882E12|
|192.26|192.26|192.26|192.26| 201.0|192.2702|1.70419038E12|
|192.23|192.23|192.23|192.23| 598.0|  192.23|1.70419068E12|
| 192.3| 192.3| 192.3| 192.3| 101.0|192.3002|1.70419146E12|
|192.15|192.15|192.14|192.14| 647.0|192.1522| 1.7041944E12|
| 192.3| 192.3| 192.3| 192.3| 510.0|192.2994|  1.704195E12|
|191.97|191.97|191.97|191.97| 507.0|191.9767|1.70419524E12|
|191.86|191.86|191.81|191.81|5403.0|  19

In [108]:
for i in range(5):
  df_5m = df_5m.withColumn(
    f"open{i}",
    lead("open", i).over(windowSpec)
)

  df_5m = df_5m.withColumn(
    f"high{i}",
    lead("high", i).over(windowSpec)
)
  df_5m = df_5m.withColumn(
    f"low{i}",
    lead("low", i).over(windowSpec)
)
  df_5m = df_5m.withColumn(
    f"close{i}",
    lead("close", i).over(windowSpec)
)
  df_5m = df_5m.withColumn(
    f"vwap{i}",
    lead("vwap", i).over(windowSpec)
)

df_5m = df_5m.withColumn(
    "BUY/SELL",
    when(lead("close", 10).over(windowSpec)>col('open'),1).otherwise(0)
)

df_5m.show()

+------+------+------+------+------+--------+-------------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+--------+
|  open|  high|   low| close|volume|    vwap|    timestamp| open0| high0|  low0|close0|   vwap0| open1| high1|  low1|close1|   vwap1| open2| high2|  low2|close2|   vwap2| open3| high3|  low3|close3|   vwap3| open4| high4|  low4|close4|   vwap4|BUY/SELL|
+------+------+------+------+------+--------+-------------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+------+------+------+------+--------+--------+
|191.61| 192.0|191.61| 192.0| 616.0|191.6739|  1.704186E12|191.61| 192.0|191.61| 192.0|191.6739| 192.2| 192.2| 192.2| 192.2|   192.2|192.25|192.25|192.25|192.25|  192.25| 192.3| 192.3| 192.3| 192.3|   192.3| 192.3| 192.3| 192.3| 192.3|   

In [109]:
for row in df_5m.tail(5):
    print(row)

Row(open=242.19, high=242.2299, low=242.19, close=242.2299, volume=3106.0, vwap=242.1921, timestamp=1735681740000.0, open0=242.19, high0=242.2299, low0=242.19, close0=242.2299, vwap0=242.1921, open1=242.11, high1=242.11, low1=242.11, close1=242.11, vwap1=242.11, open2=242.13, high2=242.13, low2=242.13, close2=242.13, vwap2=242.1319, open3=242.48, high3=242.48, low3=242.48, close3=242.48, vwap3=242.4449, open4=242.48, high4=242.48, low4=242.48, close4=242.48, vwap4=242.48, BUY/SELL=0)
Row(open=242.11, high=242.11, low=242.11, close=242.11, volume=200.0, vwap=242.11, timestamp=1735683780000.0, open0=242.11, high0=242.11, low0=242.11, close0=242.11, vwap0=242.11, open1=242.13, high1=242.13, low1=242.13, close1=242.13, vwap1=242.1319, open2=242.48, high2=242.48, low2=242.48, close2=242.48, vwap2=242.4449, open3=242.48, high3=242.48, low3=242.48, close3=242.48, vwap3=242.48, open4=None, high4=None, low4=None, close4=None, vwap4=None, BUY/SELL=0)
Row(open=242.13, high=242.13, low=242.13, clo

In [110]:
w = Window.orderBy(desc("timestamp"))   # replace with your ordering column
df_numbered = df_5m.withColumn("row_num", row_number().over(w))

# Keep everything except last 5
df_5m = df_numbered.filter("row_num > 10").drop("row_num")


In [111]:
for row in df_5m.tail(5):
    print(row)

Row(open=192.3, high=192.3, low=192.3, close=192.3, volume=250.0, vwap=192.3, timestamp=1704187020000.0, open0=192.3, high0=192.3, low0=192.3, close0=192.3, vwap0=192.3, open1=192.32, high1=192.32, low1=192.26, close1=192.26, vwap1=192.2997, open2=192.6, high2=192.6, low2=192.6, close2=192.6, vwap2=192.6, open3=192.26, high3=192.26, low3=192.26, close3=192.26, vwap3=192.2702, open4=192.23, high4=192.23, low4=192.23, close4=192.23, vwap4=192.23, BUY/SELL=0)
Row(open=192.3, high=192.3, low=192.3, close=192.3, volume=100.0, vwap=192.3, timestamp=1704186660000.0, open0=192.3, high0=192.3, low0=192.3, close0=192.3, vwap0=192.3, open1=192.3, high1=192.3, low1=192.3, close1=192.3, vwap1=192.3, open2=192.32, high2=192.32, low2=192.26, close2=192.26, vwap2=192.2997, open3=192.6, high3=192.6, low3=192.6, close3=192.6, vwap3=192.6, open4=192.26, high4=192.26, low4=192.26, close4=192.26, vwap4=192.2702, BUY/SELL=0)
Row(open=192.25, high=192.25, low=192.25, close=192.25, volume=500.0, vwap=192.25, 

In [112]:
# Example columns
excluded_col = ["timestamp","transactions",'BUY/SELL']
selected_cols = [c for c in df_5m.columns if c not in excluded_col]

feature_cols = selected_cols

# Convert the target label to numeric
df_5m = df_5m.withColumn("BUY/SELL", col("BUY/SELL").cast("integer"))

In [113]:
feature_cols

['open',
 'high',
 'low',
 'close',
 'volume',
 'vwap',
 'open0',
 'high0',
 'low0',
 'close0',
 'vwap0',
 'open1',
 'high1',
 'low1',
 'close1',
 'vwap1',
 'open2',
 'high2',
 'low2',
 'close2',
 'vwap2',
 'open3',
 'high3',
 'low3',
 'close3',
 'vwap3',
 'open4',
 'high4',
 'low4',
 'close4',
 'vwap4']

In [114]:
df_5m.show()

+--------+--------+-------+--------+-------+--------+-------------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    open|    high|    low|   close| volume|    vwap|    timestamp|   open0|   high0|   low0|  close0|   vwap0|   open1|   high1|    low1|  close1|   vwap1|   open2|   high2|    low2|  close2|   vwap2|   open3|   high3|    low3|  close3|   vwap3|   open4|   high4|    low4|  close4|   vwap4|BUY/SELL|
+--------+--------+-------+--------+-------+--------+-------------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|   242.2|   242.2|  242.2|   242.2|  413.0|   242.2|1.73567988E12|   242.2|   242.2|  242.2|   2

In [115]:
# Drop the existing 'features' column if it exists
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)
df_5m = assembler.transform(df_5m)

In [116]:
train_df, test_df = df_5m.randomSplit([0.7, 0.3], seed=42)

In [117]:
lr = LogisticRegression(featuresCol="features", labelCol="BUY/SELL")
model = lr.fit(train_df)

predictions = model.transform(test_df)
predictions.select("BUY/SELL",  "prediction", "probability").show(10, truncate=False)

+--------+----------+----------------------------------------+
|BUY/SELL|prediction|probability                             |
+--------+----------+----------------------------------------+
|1       |1.0       |[0.476384616092246,0.523615383907754]   |
|1       |1.0       |[0.2686845631771928,0.7313154368228072] |
|1       |1.0       |[0.26698816572450634,0.7330118342754937]|
|1       |1.0       |[0.14359702271986868,0.8564029772801314]|
|1       |0.0       |[0.5647004759861894,0.43529952401381056]|
|1       |1.0       |[0.27522151494297004,0.72477848505703]  |
|1       |1.0       |[0.25098122508932985,0.7490187749106701]|
|1       |0.0       |[0.5867486617386349,0.41325133826136506]|
|1       |1.0       |[0.3748432817132043,0.6251567182867956] |
|1       |0.0       |[0.6403893191926696,0.3596106808073304] |
+--------+----------+----------------------------------------+
only showing top 10 rows



In [118]:
  evalutations(predictions).show()


Accuracy: 0.7297
F1 Score: 0.7296
Precision: 0.7298
Recall: 0.7297
AUC: 0.7954
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       1|       1.0|14804|
|       1|       0.0| 5030|
|       0|       0.0|13827|
|       0|       1.0| 5576|
+--------+----------+-----+



In [119]:
#model.save("model_lr_2")

In [120]:
   !zip -r /content/model_lr_2.zip /content/model_lr_2

In [121]:
rf = RandomForestClassifier(featuresCol="features", labelCol="BUY/SELL", numTrees=100)
model = rf.fit(train_df)
predictions = model.transform(test_df)
predictions.select("BUY/SELL", "prediction", "probability").show(10, truncate=False)


+--------+----------+----------------------------------------+
|BUY/SELL|prediction|probability                             |
+--------+----------+----------------------------------------+
|1       |1.0       |[0.4866735341936226,0.5133264658063773] |
|1       |1.0       |[0.4825814298585201,0.51741857014148]   |
|1       |1.0       |[0.48220504668827807,0.517794953311722] |
|1       |1.0       |[0.48468602499709396,0.515313975002906] |
|1       |1.0       |[0.48368061077953817,0.5163193892204618]|
|1       |1.0       |[0.4825814298585201,0.51741857014148]   |
|1       |1.0       |[0.48468602499709396,0.515313975002906] |
|1       |1.0       |[0.4825814298585201,0.51741857014148]   |
|1       |1.0       |[0.4866735341936226,0.5133264658063773] |
|1       |1.0       |[0.4825814298585201,0.51741857014148]   |
+--------+----------+----------------------------------------+
only showing top 10 rows



In [122]:
evalutations(predictions).show()

Accuracy: 0.5323
F1 Score: 0.4901
Precision: 0.5433
Recall: 0.5323
AUC: 0.5561
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       1|       1.0|16184|
|       0|       1.0|14701|
|       1|       0.0| 3650|
|       0|       0.0| 4702|
+--------+----------+-----+



In [123]:
gbt = GBTClassifier(featuresCol="features", labelCol="BUY/SELL", maxIter=50)
model = gbt.fit(train_df)
predictions = model.transform(test_df)
predictions.select("BUY/SELL", "prediction", "probability").show(10, truncate=False)


+--------+----------+---------------------------------------+
|BUY/SELL|prediction|probability                            |
+--------+----------+---------------------------------------+
|1       |0.0       |[0.5231616346073723,0.4768383653926277]|
|1       |1.0       |[0.4783212325428969,0.5216787674571031]|
|1       |1.0       |[0.4783212325428969,0.5216787674571031]|
|1       |0.0       |[0.5027546765245028,0.4972453234754972]|
|1       |1.0       |[0.4940611130606785,0.5059388869393215]|
|1       |1.0       |[0.4783212325428969,0.5216787674571031]|
|1       |0.0       |[0.5027546765245028,0.4972453234754972]|
|1       |1.0       |[0.4783212325428969,0.5216787674571031]|
|1       |0.0       |[0.5027546765245028,0.4972453234754972]|
|1       |1.0       |[0.4783212325428969,0.5216787674571031]|
+--------+----------+---------------------------------------+
only showing top 10 rows



In [124]:
evalutations(predictions).show()

Accuracy: 0.5512
F1 Score: 0.5354
Precision: 0.5568
Recall: 0.5512
AUC: 0.5850
+--------+----------+-----+
|BUY/SELL|prediction|count|
+--------+----------+-----+
|       1|       0.0| 5289|
|       1|       1.0|14545|
|       0|       0.0| 7084|
|       0|       1.0|12319|
+--------+----------+-----+

